<a href="https://colab.research.google.com/github/pareshrchaudhary/numericalmethods/blob/main/Numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is all about representation of numbers on computers.

In [ ]:
import sys
import struct
import numpy as np

Note the system properties belowl. These properties will be addressed below in detail.

In [ ]:
#float information in python
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

We have 32 or 64 bits binary representation of numbers on the computer but using just binary numbers is not precise and also the range to represent the numbers is limited.

Floating point number representation is a way to use same 32 or 64 bits cleverly. It has **sign indicator** (+ or -), **exponent** (power of 2) and **fraction** (coefficient of exponent)


Python utilizes IEEE754 double precision 64 total bits to represent floats.

1 bit -> $s$ sign indicator

11 bits -> $e$ exponent ($ 2^{11} $ 2048 values)

52 bits -> $f$ fraction

To increase the precision, some of the above values must represent negative exponents i.e numbers between 0 and 1.to solve this ${e-1023}$ is done to normalize it.

To represent fractions i.e numbers between 1 and 2 the leading term will always be 1 and it is waste of bits to store it so it is dropped.

\begin{align*}
\boxed{n = (-1)^s \cdot 2^{e-1023} \cdot (1+f)}
\end{align*}




### 1) **Sign of float**

float 9.0 and -9.0 in IEEE754

In [ ]:
# Convert the float value to a bytes object containing its binary representation
binary1 = struct.pack('>f', 9.0)
binary2 = struct.pack('>f', -9.0)

# Extract the sign bit, exponent, and mantissa
sign_bit1 = (binary1[0] >> 7) & 1
exponent1 = ((binary1[0] & 0b01111111) << 1) | (binary1[1] >> 7)
mantissa1 = ((binary1[1] & 0b01111111) << 16) | (binary1[2] << 8) | binary1[3]

sign_bit2 = (binary2[0] >> 7) & 1
exponent2 = ((binary2[0] & 0b01111111) << 1) | (binary2[1] >> 7)
mantissa2 = ((binary2[1] & 0b01111111) << 16) | (binary2[2] << 8) | binary2[3]

# Combine the sign bit, exponent, and mantissa into a single binary string
binary_string1 = '{0} {1:08b} {2:023b}'.format(sign_bit1, exponent1, mantissa1)
binary_string2 = '{0} {1:08b} {2:023b}'.format(sign_bit2, exponent2, mantissa2)
print(binary_string1)
print(binary_string2)
print("^")

0 10000010 00100000000000000000000
1 10000010 00100000000000000000000
^


### 2) **Largest and smallest number close to a float**

Lets take 9.0 float from above as the float of choice. Since the number is positive sign $s = 0$. The largest power of two smaller than 9.0 is 8 so the exponent is 3, resulting in characteristic equation

$ 3 + 1023 = 1026(base10) = 10000010 (base2) $

Then the fraction part is

$\frac98 - 1 = 0.875 (base 10) = 1 \cdot \frac{1}{2^1} + \frac{1}{2^2} + \frac{1}{2^3}  = 00100000000000000000000 (base2)$

Now next smaller binary number will be 0 10000010 00011111111111111111111

In [ ]:
#Next smaller number

small = "01000001000011111111111111111111"

# Convert the binary string to a bytes object
binary = int(small, 2).to_bytes(4, byteorder='big')

# Unpack the bytes object as a float
float_value = struct.unpack('>f', binary)[0]
print(float_value)

8.999999046325684


Next largest number will be 0 10000010 00100000000000000000001



In [ ]:
#Next large number
large = "01000001000100000000000000000001"

# Convert the binary string to a bytes object
binary = int(large, 2).to_bytes(4, byteorder='big')

# Unpack the bytes object as a float
float_value = struct.unpack('>f', binary)[0]
print(float_value)

9.000000953674316


Any computation that falls in between $8.999999046325684$ and $9.000000953674316$ will be assigned 9.0.

IEEE754 number represents number 9.0 along with all the real numbers halfway between its immediate neighbors.

The distance between one number to the next is called gap. As the fraction is multiplied by $ 2^{e-1023}$, the gap grows as the number represented grows.

In [ ]:
# gap at 9.0
# exponent at 9.0 is 3
x = 9.0
gap = np.spacing(x)
print(gap)

1.7763568394002505e-15


If we take a number and add half of the gap present at that number into that number then the output will change back to the original number.

In [ ]:
x = 30.0
gap = np.spacing(x)
x = x + gap/2

print(x)

30.0


**Gap grows as the number grows.**

$ 2^{e-1023}$ grows as the number grows.

In [ ]:
x = 100.0
gap = np.spacing(x)
print(gap)

1.4210854715202004e-14


In [ ]:
x = 100000000000.0
gap = np.spacing(x)
print(gap)

1.52587890625e-05


### Breaking bounds both ways

One of the good methods in engineers is strees testing.

Lets check IEEE754 at the bundaries. These boundaries are present at e = 20

#### 1) Underflow

Underflow happens when a floating-point number becomes too small to be represented accurately.

In [ ]:
x = sys.float_info.min
print(x)

2.2250738585072014e-308


In [ ]:
small_value = x/2
y = x - small_value
print(y)

1.1125369292536007e-308


In [ ]:
small_value = x
y = x - small_value
print(y)

0.0


#### 2) Overflow

Overflow happens when a number becomes too large to be represented accurately.

In [ ]:
x = sys.float_info.max
print(x)

1.7976931348623157e+308


In [ ]:
small_value = 1e291
y = x + small_value
print(y)

1.7976931348623157e+308


In [ ]:
small_value = 1e292
y = x + small_value
print(y)

inf
